# Approximating likelihood ratios with calibrated classifiers

Gilles Louppe, January 2016.


In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import theano
import theano.tensor as T

## Toy problem

Let us consider two 1D distributions $p_0$ and $p_1$ for which we want to approximate the ratio $r(x) = \frac{p_0(x)}{p_1(x)}$ of their densities.

- $p_1$ is defined as a mixture of two gaussians;
- $p_0$ is defined as a mixture of the same two gaussians + a bump.

In [ ]:
from carl.distributions import Normal
from carl.distributions import Mixture

components = [
    Normal(mu=-2.0, sigma=0.75, random_state=0),   # c0
    Normal(mu=0.0, sigma=2.0, random_state=1),     # c1
    Normal(mu=1.0, sigma=0.5, random_state=2)      # c2 (bump)
]

bump_coefficient = 0.05
g = theano.shared(bump_coefficient) 
p0 = Mixture(components=components, weights=[0.5 - g / 2., 0.5 - g / 2., g], random_state=10)
p1 = Mixture(components=components[:2], weights=[0.5, 0.5], random_state=11)

Note: for $p_0$, weights are all tied together through the Theano shared variable `g`. This means that changes to the value stored in `g` also automatically change the weight values and the resulting mixture.

In [ ]:
X_true = p0.rvs(5000)

In [ ]:
reals = np.linspace(-5, 5, num=1000)
plt.plot(reals, p0.pdf(reals.reshape(-1, 1)), label=r"$p(x|\theta_0)$", color="b")
plt.plot(reals, p1.pdf(reals.reshape(-1, 1)), label=r"$p(x|\theta_1)$", color="r")
plt.hist(X_true[:, 0], bins=100, normed=True, label="data", alpha=0.2, color="b")
plt.xlim(-5, 5)
plt.legend(loc="best", prop={'size': 8})
plt.show()

## Density ratio estimation

The density ratio $r(x)$ can be approximated using calibrated classifiers, either directly by learning to classify $x \sim p_0$ from $x \sim p_1$, or by decomposing the ratio of the two mixtures as pairs of simpler density ratios.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

from carl.ratios import ClassifierRatio
from carl.ratios import DecomposedRatio
from carl.learning import CalibratedClassifierCV
from scipy.stats import chi2, norm

# Classifier
# from sklearn.linear_model import ElasticNetCV
# clf = ElasticNetCV()

from sklearn.neural_network import MLPRegressor
clf = MLPRegressor(activation="tanh", hidden_layer_sizes=(10, 10), random_state=0)

# from sklearn.ensemble import ExtraTreesRegressor
# clf = ExtraTreesRegressor(n_estimators=250, max_leaf_nodes=15)

n_train_samples = 100000
n_calibration_samples = 100000*5

# No calibration
cc_none = ClassifierRatio(base_estimator=clf)
cc_none.fit(numerator=p0, denominator=p1, n_samples=n_train_samples)

# Calibration + Direct approximation 
cv = StratifiedShuffleSplit(n_iter=1, test_size=0.75, random_state=0)  # 25% for training, 75% for calibration
cc_direct = ClassifierRatio(base_estimator=CalibratedClassifierCV(clf, cv=cv))
cc_direct.fit(numerator=p0, denominator=p1, n_samples=n_train_samples)

# Calibration + Decomposition of the mixture
cc_decomposed = DecomposedRatio(ClassifierRatio(base_estimator=CalibratedClassifierCV(clf, cv=cv)))
cc_decomposed.fit(numerator=p0, denominator=p1, n_samples=n_calibration_samples)

Note: `CalibratedClassifierRatio` takes three arguments for controlling its execution:
- `base_estimator` specifying the classifier to be used,
- `calibration` specifying the calibration algorithm (`"kde"`, `"histogram"`, or a user-defined distribution-like object),
- `cv` specifying how to allocate data for training and calibration.

In [ ]:
plt.plot(reals, -p0.nnlf(reals.reshape(-1, 1))  
                +p1.nnlf(reals.reshape(-1, 1)), label="True ratio")

plt.plot(reals, cc_none.predict(reals.reshape(-1, 1), log=True), label="No calibration")
plt.plot(reals, cc_direct.predict(reals.reshape(-1, 1), log=True), label="Calibration")
plt.plot(reals, cc_decomposed.predict(reals.reshape(-1, 1), log=True), label="Calibration + Decomposition")

plt.xlim(-5, 5)
plt.ylim(-0.5, 0.5)
plt.legend(loc="best", prop={'size': 8})
plt.show()

## Using density ratios for maximum likelihood fit

In the likelihood-free setting, density ratios can be used to find the maximum likelihood estimator $\theta^* = \arg \max_{\theta} p(D | \theta)$ by noticing that $\theta^*$ also maximizes $\prod_{x \in D} \frac{p(x|\theta)}{p(x|\theta_1)}$ for some fixed value of $\theta_1$.

As an example, this can be used to find the bump coefficient in $p_1$, as illustrated below:

In [ ]:
from sklearn.utils import check_random_state
from scipy.optimize import minimize

n_trials = 1000 
mles = []
nll = []

for i in range(n_trials):  
    # Reset
    rng = check_random_state(i)
    p0.set_params(random_state=rng)
    p1.set_params(random_state=rng)
    
    g.set_value(bump_coefficient)
    X_true = p0.rvs(5000)
    
    # Fit ratio
    clf = MLPRegressor(activation="tanh", hidden_layer_sizes=(10, 10), random_state=rng)
    cc = DecomposedRatio(ClassifierRatio(base_estimator=CalibratedClassifierCV(clf, cv=cv)))
    cc.fit(numerator=p0, denominator=p1, n_samples=n_calibration_samples)

    # Inference
    def objective(theta):       
        g.set_value(theta[0])  # this indirectly updates weights in p1, 
                               # along with the density ratios computed by cc 
                               # (without having to retrain the classifiers since 
                               # g only affects the weights!)

        return -np.sum(cc.predict(X_true, log=True))

    results = minimize(objective, x0=[0.1], 
                       constraints=[{'type':'ineq', 'fun': lambda x: x[0]},
                                    {'type':'ineq', 'fun': lambda x: 1. - x[0]},])

    # add NLL evaluate at true point
    nll_mle = results.fun

    g.set_value(bump_coefficient)
    nll_true = -np.sum(cc.predict(X_true, log=True))
    nll.append(2.*(nll_true - nll_mle))
    mles.append(results.x[0])
    if i%100 ==0 :
        print(i)
        print("nll: ", nll_mle, nll_true)

In [ ]:
np.mean(mles)

In [ ]:
h = plt.hist(mles, bins=30, normed=1, alpha=0.2)
plt.vlines(bump_coefficient, 0, h[0].max()+5, linestyles="dashed", label="true g")
plt.legend()
plt.show()

In [ ]:
bins = np.linspace(0,3,50)
test = np.linspace(0,3,100)
counts, bins, patches = plt.hist(2*nll, bins=bins, normed=1, alpha=0.2)
plt.plot(test, chi2.pdf(test,df=1))
plt.xlabel('-2 log L(g_true)/L(g_mle)')
#plt.ylim(1e-2,5)
#plt.semilogy()

In [ ]:
plt.plot(bins[1:], np.cumsum(counts)*(bins[1]-bins[0]), c='b')
plt.plot(bins[1:], chi2.cdf(bins[1:],df=1), c='g')
plt.xlabel('-2 log L(g_true)/L(g_mle)')

In [ ]:
counts, bins, patches = plt.hist(np.sqrt(2*nll), bins=bins, normed=1, alpha=0.2)
plt.plot(test, 2*norm.pdf(test))